In [4]:
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC

import features
import config
import data_loader
import os
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
import eli5
import nltk
from IPython.display import display
import numpy as np

seed_value = 42  # random seed of 42 for all experiments
os.environ['PYTHONHASHSEED'] = str(seed_value)
np.random.seed(seed_value)

In [5]:
def train_tfidf(train):
    vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', encoding='utf-8')
    train_sentences, test_sentences = [], []
    for x in train:
        train_sentences.append(features.get_words_str(x))
    X_train = vectorizer.fit_transform(train_sentences)
    return vectorizer, X_train


def make_predictions(annotated_files, vectorizer, svm):
    count = 0
    test_sentences = []
    filenames = []

    rootdir = '../../dataset/minNarrative_txtfiles/'
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if file not in annotated_files: # Only look at non-annotated files
                filepath = os.path.join(subdir, file)
                filenames.append(filepath)
                count += 1
                with open(filepath) as file:
                    text = file.read().strip()
                test_sentences.append(' '.join(features.filter_punct(word_tokenize(text))))
    X_test = vectorizer.transform(test_sentences)

    print("Making predictions for {} documents".format(count))
    preds_with_probs = svm.predict_proba(X_test)[:, 1]

    return preds_with_probs, filenames, X_test


def SVM_model(X_train, Y):
    tuned_parameters = [{'C': [1], 'kernel': ['linear']}, ]
    algo = SVC(probability=True)

    param_object = ParameterGrid(tuned_parameters)
    for param_dict in param_object:
        print("Running for parameters:", param_dict)
        algo.set_params(**param_dict)  # set the desired hyperparameters

        print("Training SVM TFIDF model with {} documents".format(Y.shape[0]))
        clf = algo.fit(X_train, Y)

        return clf

    
def average_probabilities(preds_with_probs, filenames):
    genre_probabilities = dict()
    for i in range(len(filenames)):
        genre = filenames[i].split('/')[-1].split('_')[0]
        if genre == 'FOX' or genre == 'CNN':
            genre = 'OPINION'

        if genre in genre_probabilities:
            genre_probabilities[genre].append(preds_with_probs[i])
        else:
            genre_probabilities[genre] = [preds_with_probs[i]]

    for k, v in genre_probabilities.items():
        avg = round(sum(v) / len(v), 3)
        print('Genre {}, {} documents, average predicted probability: {}'.format(k, len(v), avg))


In [6]:
# Loads annotated filenames and labels (334 files)
X, Y = data_loader.load_annotated_data(threshold=2.5)

# Creates Tfidf vectorizer using the annotated files
vectorizer, X_train = train_tfidf(X)

# Trains SVM model using the annotated files
svm = SVM_model(X_train, Y)

# Makes predictions on the rest of the data set (17k+ documents) using the SVM model and Tfidf Vectorizer
preds_with_probs, filenames, X_test = make_predictions(X, vectorizer, svm)

Loading annotated data from: ../../dataset/MinNarrative_ReaderData_Final.csv
Running for parameters: {'C': 1, 'kernel': 'linear'}
Training SVM TFIDF model with 334 documents
Making predictions for 17372 documents


# Average predicted probability per genre

In [7]:
average_probabilities(preds_with_probs, filenames)

Genre PHIL, 538 documents, average predicted probability: 0.216
Genre SCOTUS, 965 documents, average predicted probability: 0.132
Genre APHORISM, 467 documents, average predicted probability: 0.171
Genre OPINION, 1611 documents, average predicted probability: 0.647
Genre BIO, 973 documents, average predicted probability: 0.9
Genre FABLE, 263 documents, average predicted probability: 0.969
Genre SHORT, 488 documents, average predicted probability: 0.973
Genre REDDIT, 980 documents, average predicted probability: 0.98
Genre ROC, 977 documents, average predicted probability: 0.983
Genre BREVIEW, 864 documents, average predicted probability: 0.47
Genre NOVEL19C, 1025 documents, average predicted probability: 0.926
Genre FAIRY, 764 documents, average predicted probability: 0.943
Genre HIST, 1050 documents, average predicted probability: 0.773
Genre FLASH, 877 documents, average predicted probability: 0.952
Genre ACADEMIC-SCIENCE, 967 documents, average predicted probability: 0.086
Genre ACA

# Feature Analysis

In [8]:
# Get corresponding feature names of Tfidf Vectorizer
feature_names = list(vectorizer.get_feature_names_out())

In [9]:
# Get index of passages with lowest and highest predicted degree of narrativty
highest_degree = np.where(preds_with_probs ==np.amax(preds_with_probs))[0][0]
lowest_degree = np.where(preds_with_probs ==np.amin(preds_with_probs))[0][0]
print(filenames[highest_degree])
print(filenames[lowest_degree])

../../dataset/minNarrative_txtfiles/opinions/fox-articles/FOX_janice-dean-coronavirus-in-laws-cuomo.txt
../../dataset/minNarrative_txtfiles/legal-contracts/LEGAL_5S_0_ASIANDRAGONGROUPINC_08_11_2005-EX-10-5-Reseller-Agreement.txt


In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [15]:
def show_weights(fname):
    with open(fname) as file:
        text = file.read().strip()
        text = ' '.join(word_tokenize(text))
        display(eli5.explain_prediction(svm, text, vec=vectorizer, feature_names=feature_names, targets=['POS']))

print(preds_with_probs[lowest_degree])
print(filenames[lowest_degree])
show_weights(filenames[lowest_degree])

1.0951326855188498e-05
../../dataset/minNarrative_txtfiles/legal-contracts/LEGAL_5S_0_ASIANDRAGONGROUPINC_08_11_2005-EX-10-5-Reseller-Agreement.txt


Explanation(estimator="SVC(C=1, kernel='linear', probability=True)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='POS', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='and', weight=0.10916643162835529, std=None, value=0.1903728177698159), FeatureWeight(feature='<BIAS>', weight=0.0635826125516246, std=None, value=1.0), FeatureWeight(feature='all', weight=0.02411092829178852, std=None, value=0.16517768006674752), FeatureWeight(feature='one', weight=0.014868896412827919, std=None, value=0.06607107202669901), FeatureWeight(feature='but', weight=0.01448122111735735, std=None, value=0.02896241564294588), FeatureWeight(feature='them', weight=0.012928648663650174, std=None, value=0.022621762137507603), FeatureWeight(feature='into', weight=0.012111051789080276, std=None, value=0.039820486818614595), FeatureWeight(feature='their', weight=0.011425543537935613, std=None, value=0.019446976825532373), FeatureWeight(feature='to', weight=0.011217211399158005, std=None, value=0.07516847591232688), FeatureWeight(feature='number', weight=0.008653833063519866, std=None, value=0.03721502558461096), FeatureWeight(feature='third', weight=0.008515137245192895, std=None, value=0.072145583764422), FeatureWeight(feature='money', weight=0.008116412138738408, std=None, value=0.03272364907011785), FeatureWeight(feature='offered', weight=0.007113598131185593, std=None, value=0.040219461972094805), FeatureWeight(feature='war', weight=0.00689537786241142, std=None, value=0.03207890765606015), FeatureWeight(feature='some', weight=0.0068336199234931795, std=None, value=0.050578658135308284), FeatureWeight(feature='convenience', weight=0.006825756674648942, std=None, value=0.04535557990064562), FeatureWeight(feature='now', weight=0.006332978244327307, std=None, value=0.026640304795282798), FeatureWeight(feature='good', weight=0.0062611665117997825, std=None, value=0.026942789727509332), FeatureWeight(feature='explosion', weight=0.00621942532947714, std=None, value=0.04535557990064562), FeatureWeight(feature='declared', weight=0.005020200254737229, std=None, value=0.08470228702632356), FeatureWeight(feature='orders', weight=0.004766570190885788, std=None, value=0.08470228702632356), FeatureWeight(feature='disclosed', weight=0.004260544561851853, std=None, value=0.04535557990064562), FeatureWeight(feature='affected', weight=0.004241278360499548, std=None, value=0.03508334404354399), FeatureWeight(feature='than', weight=0.0041188920940912685, std=None, value=0.022796119637625525), FeatureWeight(feature='enemy', weight=0.00399193608834387, std=None, value=0.03508334404354399), FeatureWeight(feature='receiving', weight=0.0038778568956308493, std=None, value=0.04535557990064562), FeatureWeight(feature='dispute', weight=0.0033576416978301476, std=None, value=0.04235114351316178), FeatureWeight(feature='god', weight=0.0031709707586825547, std=None, value=0.036072791882211), FeatureWeight(feature='cost', weight=0.0031611006250222866, std=None, value=0.036072791882211), FeatureWeight(feature='both', weight=0.0031187041744975778, std=None, value=0.02580055602510937), FeatureWeight(feature='know', weight=0.003104458092778788, std=None, value=0.052699370241559275), FeatureWeight(feature='follows', weight=0.0030819208771951665, std=None, value=0.04235114351316178), FeatureWeight(feature='flood', weight=0.0030508099489062242, std=None, value=0.04535557990064562), FeatureWeight(feature='telephone', weight=0.0025543570218482486, std=None, value=0.04535557990064562), FeatureWeight(feature='processes', weight=0.002361304742837606, std=None, value=0.03721502558461096), FeatureWeight(feature='on', weight=0.0023219568204285567, std=None, value=0.013717754290251863), FeatureWeight(feature='trade', weight=0.002236734490260169, std=None, value=0.04235114351316178), FeatureWeight(feature='administration', weight=0.001717601046181095, std=None, value=0.04535557990064562), FeatureWeight(feature='labour', weight=0.00

In [17]:
print(preds_with_probs[highest_degree])
print(filenames[highest_degree])
show_weights(filenames[highest_degree])

0.9999999999999699
../../dataset/minNarrative_txtfiles/opinions/fox-articles/FOX_janice-dean-coronavirus-in-laws-cuomo.txt


Explanation(estimator="SVC(C=1, kernel='linear', probability=True)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='POS', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='he', weight=0.38905154448054824, std=None, value=0.23803165126633463), FeatureWeight(feature='was', weight=0.36589917460618326, std=None, value=0.22331011328943653), FeatureWeight(feature='his', weight=0.25851288703237785, std=None, value=0.19758371384942425), FeatureWeight(feature='she', weight=0.2235080226277401, std=None, value=0.2138225618415105), FeatureWeight(feature='had', weight=0.2014779783911754, std=None, value=0.14195208790693764), FeatureWeight(feature='her', weight=0.12362858635419788, std=None, value=0.15906459612170787), FeatureWeight(feature='and', weight=0.0903494443725379, std=None, value=0.15755830847062025), FeatureWeight(feature='they', weight=0.0846755501255866, std=None, value=0.1461283542794559), FeatureWeight(feature='we', weight=0.06907593286018072, std=None, value=0.16309959128713655), FeatureWeight(feature='my', weight=0.06412373599613314, std=None, value=0.06283981308898105), FeatureWeight(feature='<BIAS>', weight=0.0635826125516246, std=None, value=1.0), FeatureWeight(feature='them', weight=0.06002518185456983, std=None, value=0.10502840795670165), FeatureWeight(feature='him', weight=0.04912392858207725, std=None, value=0.05126102815313345), FeatureWeight(feature='over', weight=0.04446899989195178, std=None, value=0.10089920806424177), FeatureWeight(feature='home', weight=0.04122908476090463, std=None, value=0.09586742447869001), FeatureWeight(feature='you', weight=0.040873348228614804, std=None, value=0.029350538844721042), FeatureWeight(feature='went', weight=0.03860398510570763, std=None, value=0.0656812038511884), FeatureWeight(feature='their', weight=0.03713259154501479, std=None, value=0.06320195138640514), FeatureWeight(feature='were', weight=0.034722012314065145, std=None, value=0.15735979078951437), FeatureWeight(feature='would', weight=0.03214858611297183, std=None, value=0.04594445317544839), FeatureWeight(feature='years', weight=0.02926556972691614, std=None, value=0.1024668001596784), FeatureWeight(feature='to', weight=0.027775634165744866, std=None, value=0.18612933406016727), FeatureWeight(feature='at', weight=0.027334742052511542, std=None, value=0.06979401555023011), FeatureWeight(feature='but', weight=0.02689338858714714, std=None, value=0.05378672779014591), FeatureWeight(feature='last', weight=0.023843097081742173, std=None, value=0.06847673177049286), FeatureWeight(feature='house', weight=0.02275939753098342, std=None, value=0.04049612153500266), FeatureWeight(feature='it', weight=0.021989145418926364, std=None, value=0.06928656362637817), FeatureWeight(feature='do', weight=0.021321100997790072, std=None, value=0.03862432700228523), FeatureWeight(feature='there', weight=0.020915310042382687, std=None, value=0.03675556254035871), FeatureWeight(feature='husband', weight=0.020656093692586103, std=None, value=0.06911282800094459), FeatureWeight(feature='when', weight=0.020585251141288993, std=None, value=0.0487094514264853), FeatureWeight(feature='people', weight=0.020253240648702518, std=None, value=0.05650375492339061), FeatureWeight(feature='into', weight=0.01968023393247223, std=None, value=0.06470755055327652), FeatureWeight(feature='up', weight=0.017555753302143328, std=None, value=0.056140445690014536), FeatureWeight(feature='could', weight=0.01753245390400746, std=None, value=0.07439900827129042), FeatureWeight(feature='us', weight=0.017408248996993288, std=None, value=0.09486375694936273), FeatureWeight(feature='out', weight=0.01730917023478567, std=None, value=0.025905750307229122), FeatureWeight(feature='death', weight=0.01727226757109094, std=None, value=0.0465625189844809), FeatureWeight(feature='another', weight=0.01623447873798611, std=None, value=0.05478138541639429), FeatureWeight(feature='on', weight=0.015092546610471937, s

In [18]:
eli5.show_weights(svm, feature_names=feature_names)

Weight?,Feature
+1.639,was
+1.634,he
+1.419,had
+1.393,you
+1.308,his
+1.045,she
+1.020,my
+0.958,him
+0.898,said
+0.777,her
